## Setup

In [1]:
import sqlite3
import pandas as pd

# Connect to the Chinook database
conn = sqlite3.connect("Chinook_Sqlite.sqlite")

In [2]:
# Register the %sql magic
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def sql(line, cell):
    return pd.read_sql_query(cell, conn)

## Reading DB

In [67]:
%%sql
select name from sqlite_master where type='table';

,name
0,Album
1,Artist
2,Customer
3,Employee
4,Genre
5,Invoice
6,InvoiceLine
7,MediaType
8,Playlist
9,PlaylistTrack


In [68]:
%%sql
select * from Album limit 5;

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


In [69]:
%%sql
select * from Artist limit 5;

,ArtistId,Name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains


In [70]:
%%sql
select * from Customer limit 1;

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3


In [71]:
%%sql
select * from Employee limit 1;

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2002-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com


In [72]:
%%sql
select * from Genre limit 5;

,GenreId,Name
0,1,Rock
1,2,Jazz
2,3,Metal
3,4,Alternative & Punk
4,5,Rock And Roll


In [73]:
%%sql
select * from invoice limit 5;

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86


In [74]:
%%sql
select * from invoiceline limit 5;

,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity
0,1,1,2,0.99,1
1,2,1,4,0.99,1
2,3,2,6,0.99,1
3,4,2,8,0.99,1
4,5,2,10,0.99,1


In [75]:
%%sql
select * from MediaType limit 5;

,MediaTypeId,Name
0,1,MPEG audio file
1,2,Protected AAC audio file
2,3,Protected MPEG-4 video file
3,4,Purchased AAC audio file
4,5,AAC audio file


In [76]:
%%sql
select * from playlist limit 5;

,PlaylistId,Name
0,1,Music
1,2,Movies
2,3,TV Shows
3,4,Audiobooks
4,5,90’s Music


In [77]:
%%sql
select * from playlisttrack limit 5;

,PlaylistId,TrackId
0,1,3402
1,1,3389
2,1,3390
3,1,3391
4,1,3392


In [78]:
%%sql
select * from track limit 5;

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


## Answering key business questions

In [79]:
%%sql
select name as Product ,sum(Quantity) as total_quantity from invoiceline il
join track t on il.trackid = t.trackid
group by t.name
order by total_quantity desc
limit 10;

,Product,total_quantity
0,The Trooper,5
1,Untitled,4
2,The Number Of The Beast,4
3,Sure Know Something,4
4,Hallowed Be Thy Name,4
5,Eruption,4
6,Where Eagles Dare,3
7,Welcome Home (Sanitarium),3
8,Sweetest Thing,3
9,Surrender,3


In [80]:
%%sql
select BillingCountry as Country, round(sum(unitprice)*sum(Quantity),2) as Total_Sales
from invoice i
join invoiceline il on i.invoiceid = il.invoiceid
group by Country
order by Total_Sales desc
limit 10;

,Country,Total_Sales
0,USA,258391.64
1,Canada,92403.84
2,France,37069.00
3,Brazil,36119.00
4,Germany,23784.96
5,United Kingdom,12866.04
6,Czech Republic,6858.24
7,Portugal,5870.24
8,India,5569.24
9,Chile,1771.56


In [81]:
%%sql
select strftime('%Y-%m', invoicedate) as Month,
       round(sum(unitprice)*sum(Quantity),2) as Month_Sales
from invoice i
join invoiceline il on i.invoiceid = il.invoiceid
group by Month
order by Month_Sales desc
limit 10;

,Month,Month_Sales
0,2010-01,1999.56
1,2011-04,1961.56
2,2011-06,1923.56
3,2013-11,1885.56
4,2012-08,1809.56
5,2010-02,1771.56
6,2010-03,1695.56
7,2012-10,1619.56
8,2011-05,1619.56
9,2012-07,1505.56


## Bonus

In [82]:
%%sql
SELECT Country, Product, Total_unit_Sold
FROM (
    SELECT 
        i.BillingCountry AS Country,
        t.Name AS Product,
        SUM(il.Quantity) AS Total_unit_Sold,
        RANK() OVER (PARTITION BY i.BillingCountry ORDER BY SUM(il.Quantity) DESC) AS Rank
    FROM Invoice i
    JOIN InvoiceLine il ON i.InvoiceId = il.InvoiceId
    JOIN Track t ON il.TrackId = t.TrackId
    GROUP BY i.BillingCountry, t.Name
) sub
WHERE Rank = 1
ORDER BY Country, Product;

,Country,Product,Total_unit_Sold
0,Argentina,Battery,1
1,Argentina,Better Than You,1
2,Argentina,Cabeça Dinossauro,1
3,Argentina,Canário Do Reino,1
4,Argentina,Chemical Wedding,1
...,...,...,...
917,Spain,W.M.A.,1
918,Spain,We Are The Champions,1
919,Sweden,Polly,2
920,USA,Heaven Can Wait,3
